# Computational Graph Implementation
This notebook implements a `Node` class that represents values in a computational graph, supports element-wise arithmetic operations, and performs automatic differentiation using backpropagation.

In [ ]:
import numpy as np

# Node class represents a value in a computational graph and supports automatic differentiation
class Node:
    def __init__(self, value, op=None, op_args=(), children=()):
        """
        Initialize a Node in the computational graph.
        
        Parameters:
        - value (float or np.array): The numerical value stored in this node.
        - op (str, optional): The operation that created this node (e.g., 'add', 'mul').
        - op_args (tuple, optional): Additional arguments for the operation.
        - children (tuple, optional): The dependent nodes that were used to compute this node.
        """
        self.value = np.array(value, dtype=float)  # Convert to NumPy array for consistency
        self.op = op  # Store the operation
        self.op_args = op_args  # Store additional operation arguments
        self.children = children  # Track dependencies in the computation graph
        self.grad = np.zeros_like(self.value)  # Initialize gradient storage
    
    def backward(self, grad=None):
        """
        Compute gradients using backpropagation and the chain rule.
        
        Parameters:
        - grad (np.array, optional): The gradient from the previous computation.
        """
        if grad is None:
            grad = np.ones_like(self.value)  # Initialize gradient for root node
        
        self.grad += grad  # Accumulate gradients
        
        # Apply chain rule based on the operation performed
        if self.op == "add":
            self.children[0].backward(grad)
            self.children[1].backward(grad)
        elif self.op == "sub":
            self.children[0].backward(grad)
            self.children[1].backward(-grad)
        elif self.op == "mul":
            self.children[0].backward(grad * self.children[1].value)
            self.children[1].backward(grad * self.children[0].value)
        elif self.op == "div":
            self.children[0].backward(grad / self.children[1].value)
            self.children[1].backward(-grad * self.children[0].value / (self.children[1].value ** 2))
        elif self.op == "pow":
            exponent = self.op_args[0]
            self.children[0].backward(grad * exponent * (self.children[0].value ** (exponent - 1)))
    
    # Overloading arithmetic operators to support computational graph
    def __add__(self, other):
        """Element-wise addition of two Nodes."""
        return Node(self.value + other.value, children=(self, other), op="add")
    
    def __sub__(self, other):
        """Element-wise subtraction of two Nodes."""
        return Node(self.value - other.value, children=(self, other), op="sub")
    
    def __mul__(self, other):
        """Element-wise multiplication of two Nodes."""
        return Node(self.value * other.value, children=(self, other), op="mul")
    
    def __truediv__(self, other):
        """Element-wise division of two Nodes."""
        return Node(self.value / other.value, children=(self, other), op="div")
    
    def __pow__(self, exponent):
        """Element-wise exponentiation of a Node."""
        return Node(self.value ** exponent, children=(self,), op="pow", op_args=(exponent,))
    
    def __repr__(self):
        """String representation of a Node."""
        return f"Node(value={self.value}, grad={self.grad})"
